# Basic RAG
Retrieval-augmented generation (RAG) is an AI framework that synergizes the capabilities of LLMs and information retrieval systems. It’s useful to answer questions or generate content leveraging external knowledge. There are two main steps in RAG: 1) retrieval: retrieve relevant information from a knowledge base with text embeddings stored in a vector store; 2) generation: insert the relevant information to the prompt for the LLM to generate information. In this guide, we will walk through a very basic example of RAG with four implementations:

- RAG from scratch with Mistral
- RAG with Mistral and LangChain
- RAG with Mistral and LlamaIndex
- RAG with Mistral and Haystack

## RAG from scratch

This section aims to guide you through the process of building a basic RAG from scratch. We have two goals: firstly, to offer users a comprehensive understanding of the internal workings of RAG and demystify the underlying mechanisms; secondly, to empower you with the essential foundations needed to build an RAG using the minimum required dependencies.


### Import needed packages
The first step is to install the needed packages `mistralai` and `faiss-cpu` and import the needed packages:



In [ ]:
#https://docs.mistral.ai/guides/rag/
! pip install faiss-cpu==1.7.4 mistralai chardet

In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import requests
import numpy as np
import faiss
import os
from getpass import getpass

api_key= 'ABC' #getpass("Type your API Key")
client = MistralClient(api_key=api_key)

In [ ]:
from mistralai.client import MistralClient

client = MistralClient(api_key="ABC")

embeddings_batch_response = client.embeddings(
    model="mistral-embed",
    input=["Embed this sentence.", "As well as this one."],
)


In [ ]:
len(embeddings_batch_response.data[0].embedding)

1024

In [ ]:
from google.colab import files
files.upload()

Saving CS_Support_FAQ.txt to CS_Support_FAQ (1).txt


{'CS_Support_FAQ (1).txt': b'### Year 2022:\r\n1. **Q:** How do I place an order?\r\n- **A:** Open the app, browse the menu, select items, add to cart, and proceed to checkout.\r\n2. **Q:** What payment methods are accepted?\r\n- **A:** We accept credit/debit cards, UPI, and net banking.\r\n3. **Q:** Can I schedule an order for later?\r\n- **A:** Yes, you can schedule your order while checking out.\r\n4. **Q:** How can I track my order?\r\n- **A:** Go to the "My Orders" section and view the status of your order.\r\n5. **Q:** Do you deliver to my area?\r\n- **A:** Enter your location in the app to see if we deliver to your area.\r\n6. **Q:** Is there a minimum order value for delivery?\r\n- **A:** Yes, the minimum order value is $5.\r\n7. **Q:** How do I apply a discount code?\r\n- **A:** Enter the promo code at the checkout page to apply your discount.\r\n8. **Q:** Can I modify my order after placing it?\r\n- **A:** You can modify your order before the restaurant starts preparing it.\r

In [ ]:
import chardet


input_filename = 'CS_Support_FAQ.txt'
output_filename = 'output.txt'

def convert_to_plain_text(input_filename, output_filename):
    # Detect the encoding of the input file
    with open(input_filename, 'rb') as f:
        encoding = chardet.detect(f.read())['encoding']

    # Open the input file with the detected encoding
    with open(input_filename, 'r', encoding=encoding) as infile:
        content = infile.read()

    # Write the content to the output file without specifying encoding
    with open(output_filename, 'w') as outfile:
        outfile.write(content)

convert_to_plain_text(input_filename, output_filename)

### Get data

In this very simple example, we are getting data from an essay written by Paul Graham:

In [ ]:
#response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
#text = response.text
!ls

'CS_Support_FAQ (1).txt'   CS_Support_FAQ.txt   output.txt   sample_data


We can also save the essay in a local file:

In [ ]:
with open('output.txt', 'r', encoding='latin-1') as f:
    text = f.read()

In [ ]:
len(text)

14995

## Split document into chunks

In a RAG system, it is crucial to split the document into smaller chunks so that it’s more effective to identify and retrieve the most relevant information in the retrieval process later. In this example, we simply split our text by character, combine 2048 characters into each chunk, and we get 37 chunks.

In [ ]:
chunk_size = 4096
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

In [ ]:
len(chunks)

4

#### Considerations:
- **Chunk size**: Depending on your specific use case, it may be necessary to customize or experiment with different chunk sizes and chunk overlap to achieve optimal performance in RAG. For example, smaller chunks can be more beneficial in retrieval processes, as larger text chunks often contain filler text that can obscure the semantic representation. As such, using smaller text chunks in the retrieval process can enable the RAG system to identify and extract relevant information more effectively and accurately.  However, it’s worth considering the trade-offs that come with using smaller chunks, such as increasing processing time and computational resources.
- **How to split**: While the simplest method is to split the text by character, there are other options depending on the use case and document structure. For example, to avoid exceeding token limits in API calls, it may be necessary to split the text by tokens. To maintain the cohesiveness of the chunks, it can be useful to split the text by sentences, paragraphs, or HTML headers. If working with code, it’s often recommended to split by meaningful code chunks for example using an Abstract Syntax Tree (AST) parser.


### Create embeddings for each text chunk
For each text chunk, we then need to create text embeddings, which are numeric representations of the text in the vector space. Words with similar meanings are expected to be in closer proximity or have a shorter distance in the vector space.
To create an embedding, use Mistral’s embeddings API endpoint and the embedding model `mistral-embed`. We create a `get_text_embedding` to get the embedding from a single text chunk and then we use list comprehension to get text embeddings for all text chunks.


In [ ]:
def get_text_embedding(input):
    embeddings_batch_response = client.embeddings(
          model="mistral-embed",
          input=input
      )
    return embeddings_batch_response.data[0].embedding

In [ ]:
for chunk in chunks:
  print(chunk)

### Year 2022:
1. **Q:** How do I place an order?
- **A:** Open the app, browse the menu, select items, add to cart, and proceed to checkout.
2. **Q:** What payment methods are accepted?
- **A:** We accept credit/debit cards, UPI, and net banking.
3. **Q:** Can I schedule an order for later?
- **A:** Yes, you can schedule your order while checking out.
4. **Q:** How can I track my order?
- **A:** Go to the "My Orders" section and view the status of your order.
5. **Q:** Do you deliver to my area?
- **A:** Enter your location in the app to see if we deliver to your area.
6. **Q:** Is there a minimum order value for delivery?
- **A:** Yes, the minimum order value is $5.
7. **Q:** How do I apply a discount code?
- **A:** Enter the promo code at the checkout page to apply your discount.
8. **Q:** Can I modify my order after placing it?
- **A:** You can modify your order before the restaurant starts preparing it.
9. **Q:** How do I cancel my order?
- **A:** You can cancel your order within 

In [ ]:
#text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])
import time

text_embeddings = []
for chunk in chunks:
    text_embeddings.append(get_text_embedding(chunk))
    time.sleep(1)
text_embeddings = np.array(text_embeddings)

In [ ]:
text_embeddings.shape

(4, 1024)

In [ ]:
text_embeddings

array([[-0.00440598,  0.03356934,  0.03118896, ..., -0.02406311,
         0.00150681, -0.00355339],
       [-0.00426483,  0.04437256,  0.03031921, ..., -0.00618362,
         0.00583649,  0.01377869],
       [-0.01902771,  0.03201294,  0.03585815, ..., -0.01713562,
         0.01415253, -0.00125027],
       [-0.0141983 ,  0.03488159,  0.04699707, ..., -0.00625229,
         0.00530624,  0.00389671]])

### Load into a vector database
Once we get the text embeddings, a common practice is to store them in a vector database for efficient processing and retrieval. There are several vector database to choose from. In our simple example, we are using an open-source vector database Faiss, which allows for efficient similarity search.  

With Faiss, we instantiate an instance of the Index class, which defines the indexing structure of the vector database. We then add the text embeddings to this indexing structure.


In [ ]:
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

#### Considerations:
- **Vector database**: When selecting a vector database, there are several factors to consider including speed, scalability, cloud management, advanced filtering, and open-source vs. closed-source.

### Create embeddings for a question
Whenever users ask a question, we also need to create embeddings for this question using the same embedding models as before.


In [ ]:
question = "What is cancellation order procedure?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape

(1, 1024)

In [ ]:
question_embeddings

array([[ 0.01326752,  0.02323914,  0.03015137, ...,  0.00584412,
        -0.02120972, -0.00757599]])

#### Considerations:
- Hypothetical Document Embeddings (HyDE): In some cases, the user’s question might not be the most relevant query to use for identifying the relevant context. Instead, it maybe more effective to generate a hypothetical answer or a hypothetical document based on the user’s query and use the embeddings of the generated text to retrieve similar text chunks.

### Retrieve similar chunks from the vector database
We can perform a search on the vector database with `index.search`, which takes two arguments: the first is the vector of the question embeddings, and the second is the number of similar vectors to retrieve. This function returns the distances and the indices of the most similar vectors to the question vector in the vector database. Then based on the returned indices, we can retrieve the actual relevant text chunks that correspond to those indices.


In [ ]:
D, I = index.search(question_embeddings, k=2)
print(I)

[[0 1]]


In [ ]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

['### Year 2022:\n1. **Q:** How do I place an order?\n- **A:** Open the app, browse the menu, select items, add to cart, and proceed to checkout.\n2. **Q:** What payment methods are accepted?\n- **A:** We accept credit/debit cards, UPI, and net banking.\n3. **Q:** Can I schedule an order for later?\n- **A:** Yes, you can schedule your order while checking out.\n4. **Q:** How can I track my order?\n- **A:** Go to the "My Orders" section and view the status of your order.\n5. **Q:** Do you deliver to my area?\n- **A:** Enter your location in the app to see if we deliver to your area.\n6. **Q:** Is there a minimum order value for delivery?\n- **A:** Yes, the minimum order value is $5.\n7. **Q:** How do I apply a discount code?\n- **A:** Enter the promo code at the checkout page to apply your discount.\n8. **Q:** Can I modify my order after placing it?\n- **A:** You can modify your order before the restaurant starts preparing it.\n9. **Q:** How do I cancel my order?\n- **A:** You can cance

#### Considerations:
- **Retrieval methods**: There are a lot different retrieval strategies. In our example, we are showing a simple similarity search with embeddings. Sometimes when there is metadata available for the data, it’s better to filter the data based on the metadata first before performing similarity search. There are also other statistical retrieval methods like TF-IDF and BM25 that use frequency and distribution of terms in the document to identify relevant text chunks.
- **Retrieved document**: Do we always retrieve individual text chunk as it is? Not always.
    - Sometimes, we would like to include more context around the actual retrieved text chunk. We call the actual retrieve text chunk “child chunk” and our goal is to retrieve a larger “parent chunk” that the “child chunk” belongs to.
    - On occasion, we might also want to provide weights to our retrieve documents. For example, a time-weighted approach would help us retrieve the most recent document.
    - One common issue in the retrieval process is the “lost in the middle” problem where the information in the middle of a long context gets lost. Our models have tried to mitigate this issue. For example, in the passkey task, our models have demonstrated the ability to find a "needle in a haystack" by retrieving a randomly inserted passkey within a long prompt, up to 32k context length. However, it is worth considering experimenting with reordering the document to determine if placing the most relevant chunks at the beginning and end leads to improved results.
  
### Combine context and question in a prompt and generate response

Finally, we can offer the retrieved text chunks as the context information within the prompt. Here is a prompt template where we can include both the retrieved text and user question in the prompt.



In [ ]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [ ]:
def run_mistral(user_message, model="mistral-medium-latest"):
    messages = [
        ChatMessage(role="user", content=user_message)
    ]
    chat_response = client.chat(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

In [ ]:
run_mistral(prompt)

'To cancel your order, you can do so within the app before the restaurant starts preparing it. Here are the steps:\n\n1. Open the app and go to the "My Orders" section.\n2. Find the order you wish to cancel and tap on it.\n3. Look for the "Cancel Order" button and click on it.\n4. Confirm the cancellation when prompted.\n\nPlease note that cancellation is free if done before the restaurant starts preparing your order. If the restaurant has already started preparing your order, you may not be able to cancel it or you may be charged a cancellation fee.'

In [ ]:
!ls

'CS_Support_FAQ (1).txt'   CS_Support_FAQ.txt   output.txt   sample_data


In [ ]:
#https://docs.mistral.ai/guides/rag/
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

# Load data
loader = TextLoader("output.txt")
docs = loader.load()
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
# Define the embedding model
embeddings = MistralAIEmbeddings(model="mistral-embed", mistral_api_key=api_key)
# Create the vector store
vector = FAISS.from_documents(documents, embeddings)
# Define a retriever interface
retriever = vector.as_retriever()
# Define LLM
model = ChatMistralAI(mistral_api_key=api_key)
# Define prompt template
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "How to cancel order ?"})
print(response["answer"])

/usr/local/lib/python3.10/dist-packages/langchain_mistralai/embeddings.py:105: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


To cancel an order, you can do so within the app before the restaurant starts preparing it. Unfortunately, you cannot change the payment method once the order is placed. If the restaurant cancels your order, you will receive a full refund. To apply for a refund, contact customer support through the app. Refunds are processed within 5-7 business days.


#### Considerations:
- Prompting techniques: Most of the prompting techniques can be used in developing a RAG system as well. For example, we can use few-shot learning to guide the model’s answers by providing a few examples. Additionally, we can explicitly instruct the model to format answers in a certain way.


In the next sections, we are going to show you how to do a similar basic RAG with some of the popular RAG frameworks. We will start with LlamaIndex and add other frameworks in the future.


## LlamaIndex

In [ ]:
!ls

'CS_Support_FAQ (1).txt'   CS_Support_FAQ.txt   output.txt   sample_data


In [ ]:
import chardet


input_filename = 'CS_Support_FAQ.txt'
output_filename = 'output.txt'

def convert_to_plain_text(input_filename, output_filename):
    # Detect the encoding of the input file
    with open(input_filename, 'rb') as f:
        encoding = chardet.detect(f.read())['encoding']

    # Open the input file with the detected encoding
    with open(input_filename, 'r', encoding=encoding) as infile:
        content = infile.read()

    # Write the content to the output file without specifying encoding
    with open(output_filename, 'w') as outfile:
        outfile.write(content)

convert_to_plain_text(input_filename, output_filename)

In [ ]:
!ls

'CS_Support_FAQ (1).txt'   CS_Support_FAQ.txt   output.txt   sample_data


In [ ]:
!ls

CS_Support_FAQ.txt  sample_data


In [ ]:
!pip install llama-index llama-index-llms-mistralai llama-index-embeddings-mistralai

In [ ]:
import os
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding

# Load data
reader = SimpleDirectoryReader(input_files=["output.txt"])
documents = reader.load_data()
# Define LLM and embedding model
Settings.llm = MistralAI(model="mistral-medium", api_key=api_key)
Settings.embed_model = MistralAIEmbedding(model_name='mistral-embed', api_key=api_key)
# Create vector store index
index = VectorStoreIndex.from_documents(documents)
# Create query engine
query_engine = index.as_query_engine(similarity_top_k=2)
response = query_engine.query(
    "How to cancel an order?"
)
print(str(response))

You can cancel your order within the app before the restaurant starts preparing it.
